In [1]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sentence_transformers import SentenceTransformer, util
import xml.etree.ElementTree as ET
import numpy as np

Get article mapping to achieve article alignment

In [3]:
df = pd.read_excel('data/category_configuration_09-08-2022_08-08-01.xlsx', sheet_name = 'article_names_matching')
title_dict = dict(zip(df["Article Title"], df["Category 2"]))
title_dict.get("application")

'Non-derogation'

Help functions to get article alignments

In [48]:
def get_article_text(article):
    
    text = ""

    for child in article:
        text += " "+child.text
    
    return text

def article_alignment(doc1_path, doc2_path):

    doc1, doc2 = ET.parse(doc1_path), ET.parse(doc2_path)
    root1, root2 = doc1.getroot(), doc2.getroot()
    try:
        body1, body2 = root1[1][2], root2[1][2]
    except:
        return
    num_pages = max(len(root1[1][2]), len(root2[1][2]))
    model = SentenceTransformer('distilbert-base-nli-mean-tokens')
    #model = AutoModelForSequenceClassification.from_pretrained("roberta-large-mnli")
    article_dict = {}

    #if False:
    if "title" in body1[0].attrib and "title" in body2[0].attrib:
    #title included in article attri
        for article1 in body1:

            

            for article2 in body2:

                title1 = article1.get("title")
                title2 = article2.get("title")

                if title_dict.get(title1.lower()) == title_dict.get(title2.lower()):

                    text1_embd = model.encode(get_article_text(article1))
                    text2_embd = model.encode(get_article_text(article2))
                    scores = util.cos_sim(text1_embd, text2_embd)

                    if max(scores[0])>0.7:
                        article_dict[article1.get("num")] = article2.get("num")

    else:
    #title not included in article attri, use sentence similarity instead

        for article1 in body1:
            
            text1_embd = model.encode(get_article_text(article1))
            text2_list_embd = model.encode([get_article_text(article2) for article2 in body2])
            scores = util.cos_sim(text1_embd, text2_list_embd)
            maxi = max(scores[0])
            print(maxi)

            if maxi > 0.7:

                index = np.argmax(scores[0])
                article_dict[article1.get("num")] = body2[index].get("num")



    return article_dict

Extract similar sentences from two documents

In [56]:
def extract_similar(doc1_path, doc2_path):

    #First get article alignments insides two documents
    article_dict = article_alignment(doc1_path, doc2_path)
    doc1, doc2 = ET.parse(doc1_path), ET.parse(doc2_path)
    root1, root2 = doc1.getroot(), doc2.getroot()
    try:
        body1, body2 = root1[1][2], root2[1][2]
    except:
        return
    num_pages = max(len(root1[1][2]), len(root2[1][2]))
    model = SentenceTransformer('distilbert-base-nli-mean-tokens')

    for article1 in body1:

        article2_num = article_dict.get(article1.get("num"))
        print("Current document match: ")
        print(article1.get('num') + '   :   ' + article2_num)
        if article2_num:
        #if current article have alignment in anthoer document

            article2 = body2.find(".//div[@num='" + article2_num + "']")
            article2_sents = []

            for child in article2:

                article2_sents.append(child.text)

            sentences2_embd = model.encode(article2_sents)

            for sentence1 in article1:
                
                sentence1_embd = model.encode(sentence1.text)
                scores = util.cos_sim(sentence1_embd, sentences2_embd)
                maxi = max(scores[0])

                if maxi < 0.95:
                    index = np.argmax(scores[0])
                    print(sentence1.text)
                    print(maxi)
                    print(article2[index].text)
                    print("//////////////////////////////////////////////////////////////")
            
            return




    



In [57]:
extract_similar('data/full data/t1989-9-canada-russian-federation-bit-1989.xml', 'data/full data/t1990-14-canada-czech-republic-bit-1990.xml')

Current document match: 
I   :   I
For the purpose of this Agreement:
tensor(1.0000)
For the purpose of this Agreement:
//////////////////////////////////////////////////////////////
(a) The term "territory" means the territory of Canada or the territory of the Union of Soviet Socialist Republics respectively, as well as those maritime areas, including the seabed and subsoil adjacent to the outer limit of the territorial sea of either of the above territories, over which the State concerned exercises, in accordance with international law, sovereign rights for the purpose of exploration and exploitation of the natural resources of such areas;
tensor(0.9311)
(i) In respect of Canada, the territory of Canada, as well as those maritime areas, including the seabed and subsoil adjacent to the outer limit of the territorial sea, over which Canada exercises, in accordance with international law, sovereign rights for the purpose of exploration and exploitation of the natural resources of such a